In [6]:
%load_ext autoreload
%autoreload 2
from synthpop.recipes.starter6 import Starter
starter_ver='6'

from output_verify import output_verify
from synthpop.synthesizer import synthesize_all, enable_logging 
import pandas as pd
import os,time
import pickle

outputfolder='/home/da/tools/synthpop/demos/outputs/'

# Synthesize by county 

In [ ]:
#for whole county
time0=time.time()
results=[]

def synthesize_counties(counties):

    for county in counties:
        phases=3
        df_check=None
        for phase in range(phases):
            print "\n county = ",county," phase = ", phase, "\n"
            
            pathout=outputfolder+county+"_starter%s_"%(starter_ver) #output folder
            
            starter = Starter(os.environ["CENSUS"], "MI", county, phase=phase)
            
            households, persons, fit, marginals=synthesize_all(starter)

            households.index.name='hh_id' #result[0] is hh
            #households['serialno'] = households['serialno'].map(lambda x: '%f' % x)
            #persons['serialno'] = persons['serialno'].map(lambda x: '%f' % x)
            #results[0].to_csv(outputfolder+county+"_starter%s_"%('3')+"HHs.csv",float_format='%.f') #float_format, not use scentific, pplies to all fp numbers 
            #could use results['serialno'] = results['serialno'].map(lambda x: '%f' % x) to change output format for serialno only
            
            #HHs and Persons output
            if phase == phases-1:
                households.to_csv(pathout+'_households.csv')
                persons.to_csv(pathout+'_persons.csv')

            df_check=output_verify(households,persons,marginals)
            df_check[0].set_index(['cat_value'], append=True,inplace=True)
            df_check[1].set_index(['cat_value'], append=True,inplace=True)
            df_check[2].set_index(['state','county','tract','block group','cat_name','hh_size'],inplace=True)
            for j in range(3):
                df_check[j].rename(columns={col:col+"_"+str(phase) for col in df_check[j].columns}, inplace=True)
            
            
            if phase == 0:
                HH_check, Person_check, HHsize_check = df_check[0] , df_check[1], df_check[2]
            else:
                HH_check=pd.merge(HH_check, df_check[0], left_index=True, right_index=True, how='left')
                Person_check=pd.merge(Person_check, df_check[1], left_index=True, right_index=True, how='left')
                HHsize_check=pd.merge(HHsize_check, df_check[2], left_index=True, right_index=True, how='left')

            if phase == phases-1:
                HH_check.to_csv(pathout+"_check_households.csv")
                Person_check.to_csv(pathout+"_check_persons.csv")
                HHsize_check.to_csv(pathout+"_check_hh_size.csv")
            if phase == 0:
                df_check[3].to_csv("hh7_persons_drawing.csv")
            if phase == 1:
                #df_ttlpersons=pd.DataFrame(data=[totalpersons],columns=['hh_size_person_factor'])
                df_check[4].to_csv("hh_size_person_factor.csv")
            if phase == 2:
                os.rename("hh7_persons_drawing.csv", pathout+"_hh7_persons_drawing.csv")
                os.rename("hh_size_person_factor.csv", pathout+"_hh_size_person_factor.csv")

            households,persons,marginals=None, None, None

            with open(pathout+'_fit_stats.pkl', 'wb') as f:
                pickle.dump(fit, f)
            print time.time()-time0
    return
    #return households, persons, fit, marginals
#synthesize_counties(["Livingston County","Oakland County","Washtenaw County","Wayne County","Monroe County","Macomb County","St. Clair County"])
synthesize_counties(["St. Clair County"])
print 'total time', time.time()-time0

# Synthesize by selected BGs

In [9]:
###for selected BGs
#enable_logging()
time0=time.time()
geoname="problembgs2.csv"
pathout=outputfolder+geoname+"_starter%s_"%(starter_ver)

dfbg=pd.read_csv(geoname,index_col=None,dtype=str) #make sure csv column in text not number, so read as'099' not 99
indd=[ind for i, ind in dfbg.iterrows()]
phases=3
df_check=None
for phase in range(phases):
    print "\nphase = ", phase, "\n"
    starter = Starter(os.environ["CENSUS"], "MI", 'Macomb County', phase=phase)

    households, persons, fit, marginals = synthesize_all(starter, indexes=indd)
    #print fit
    
    #HHs and Persons output
    if phase == phases-1:
        households.to_csv(pathout+'_households.csv')
        persons.to_csv(pathout+'_persons.csv')
    
    df_check=output_verify(households,persons,marginals)
    df_check[0].set_index(['cat_value'], append=True,inplace=True)
    df_check[1].set_index(['cat_value'], append=True,inplace=True)
    df_check[2].set_index(['state','county','tract','block group','cat_name','hh_size'],inplace=True)
    for j in range(3):
        df_check[j].rename(columns={col:col+"_"+str(phase) for col in df_check[j].columns}, inplace=True)
                
    if phase == 0:
        HH_check, Person_check, HHsize_check = df_check[0] , df_check[1], df_check[2]
    else:
        HH_check=pd.merge(HH_check, df_check[0], left_index=True, right_index=True, how='left')
        Person_check=pd.merge(Person_check, df_check[1], left_index=True, right_index=True, how='left')
        HHsize_check=pd.merge(HHsize_check, df_check[2], left_index=True, right_index=True, how='left')
      
    if phase == phases-1:
        HH_check.to_csv(pathout+"_check_households.csv")
        Person_check.to_csv(pathout+"_check_persons.csv")
        HHsize_check.to_csv(pathout+"_check_hh_size.csv")
    if phase == 0:
        df_check[3].to_csv("hh7_persons_drawing.csv")
    if phase == 1:
        #df_ttlpersons=pd.DataFrame(data=[totalpersons],columns=['hh_size_person_factor'])
        df_check[4].to_csv("hh_size_person_factor.csv")
    if phase == 2:
        os.rename("hh7_persons_drawing.csv", pathout+"_hh7_persons_drawing.csv")
        os.rename("hh_size_person_factor.csv", pathout+"_hh_size_person_factor.csv")
        
    households,persons,marginals=None, None, None
print time.time()-time0


phase =  0 

Synthesizing at geog level: 'block_group' (number of geographies is 627)
progress: 0.0% Synthesizing [state, county, tract, block group]: ['26' '099' '232300' '3'] Drawing 6 households
progress: 0.2% Synthesizing [state, county, tract, block group]: ['26' '099' '258000' '3'] Drawing 318 households
progress: 0.3% Synthesizing [state, county, tract, block group]: ['26' '099' '206700' '1'] Drawing 749 households
progress: 0.5% Synthesizing [state, county, tract, block group]: ['26' '099' '210000' '1'] Drawing 722 households
progress: 0.6% Synthesizing [state, county, tract, block group]: ['26' '099' '206700' '2'] Drawing 783 households
progress: 0.8% Synthesizing [state, county, tract, block group]: ['26' '099' '210000' '2'] Drawing 737 households
progress: 1.0% Synthesizing [state, county, tract, block group]: ['26' '099' '211000' '1'] Drawing 722 households
progress: 1.1% Synthesizing [state, county, tract, block group]: ['26' '099' '211000' '2'] Drawing 467 households
pro

output_verify.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfhh7size['hh_size_weight_7']=dfhh7size['output_persons']/dfhh7size['output_households']


In [ ]:
HH_check

In [ ]:
a==None

In [ ]:
str(indd[0].values)

In [ ]:
import numpy as np
import math
math.isnan(np.NaN)

In [ ]:
float('1.2')